# COVID-19 Dynamics

There are many visiualizations of the ongoing [COVID-19](https://en.wikipedia.org/wiki/Coronavirus_disease_2019) (or Corona) virus outbreak including this very popular [ARCGIS one](https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6). As most are static, this one specifically aims to illustrate the dynamics of the COVID-19 virus infection using the official datasets available at https://github.com/CSSEGISandData/COVID-19. It builds on [Jupyter](https://jupyter.org), [IPyLeaflet](https://github.com/jupyter-widgets/ipyleaflet), [Pandas](https://pandas.pydata.org/), [Voilà](https://github.com/voila-dashboards/voila) plus a little [ReportLab](https://reportlab.com), and running on [MyBinder](https://mybinder.org), too.

In [ ]:
from functools import partial
from math import pi

from ipyleaflet import basemap_to_tiles, basemaps, CircleMarker, \
    FullScreenControl, LayersControl, LayerGroup, Map, Marker, \
    Popup, WidgetControl
from ipywidgets import IntSlider, HBox, HTML, jslink, Layout, Output, \
    Play, Image
import pandas as pd
from reportlab.graphics.barcode.qr import QrCodeWidget
from reportlab.graphics.shapes import Drawing
from reportlab.graphics import renderPM

In [ ]:
url = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/"
       "master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
df_confirmed = pd.read_csv(url)
df_confirmed["Province/State"].fillna("", inplace=True)

In [ ]:
# df_confirmed.describe()

In [ ]:
# df_confirmed.head()

In [ ]:
provinces = df_confirmed["Province/State"]
countries = df_confirmed["Country/Region"]
locations = list(zip(df_confirmed.Lat, df_confirmed.Long))
day_cols = [col for col in df_confirmed.columns if col.count("/") == 2]

In [ ]:
print(f"Data range: {day_cols[0]} – {day_cols[-1]}.")

In [ ]:
def radius_sphere(volume):
    return (volume / pi / 4 * 3) ** (1/2.75)

In [ ]:
def create_qrcode(value, size=50):
    d = Drawing(size, size)
    qr = QrCodeWidget(value=value, barWidth=size, barHeight=size)
    d.add(qr)
    return Image(
        value=renderPM.drawToString(d, fmt="png"),
        format='png', width=size, height=size,
    )

In [ ]:
def slider_changed(change, the_map=None, output=None, slider=None, group=None):
    day = change['new']
    values = df_confirmed[day_cols[day]]
    if slider:
        slider.description = day_cols[day]
    if the_map:
        circle_layers = list(l for l in group.layers if type(l) == CircleMarker)
        if not circle_layers:
            # if output:
            #     with output:
            #         print(f"updating {day_cols[day]}")
            markers = []
            for i, loc in enumerate(locations):
                place = countries[i] if not provinces[i] else f"{provinces[i]}, {countries[i]}"
                rad = int(radius_sphere(values[i]))
                marker = CircleMarker(
                    location=tuple(loc),
                    radius=rad,
                    weight=0,
                    color="red" if rad > 0 else "white",
                    opacity=rad > 0)
                message = HTML(value = f"<b>{day_cols[day]}: {values[i]} confirmed in {place}</b>",
                    # placeholder = "Some HTML",
                    # description = "Some HTML"
                )
                marker.popup = message
                markers.append(marker)
            group.layers = tuple(markers)
        else:
            for i, marker in enumerate(circle_layers):
                place = countries[i] if not provinces[i] else f"{provinces[i]}, {countries[i]}"
                rad = int(radius_sphere(values[i]))
                marker.radius = rad
                marker.color="red" if rad > 0 else "white"
                marker.opacity = rad > 0
                marker.weight = 0
                marker.popup.value = f"<b>{day_cols[day]}: {values[i]} confirmed in {place}</b>"

In [ ]:
# output = Output()
# display(output)

m = Map(zoom=2, basemap=basemaps.CartoDB.Positron)

m += FullScreenControl()

# dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
# m.add_layer(dark_matter_layer)
# nat_geo_world_layer = basemap_to_tiles(basemaps.Esri.NatGeoWorldMap)
# m.add_layer(nat_geo_world_layer)
layers_control = LayersControl(position='topright')
m += layers_control

confirmed_group = LayerGroup(layers=[], name="Confirmed")
m += confirmed_group

play = Play(
    value=0,
    min=0,
    max=len(day_cols)-1,
    step=1,
    interval=500,
    # description="Press play",
    disabled=False
)
day_slider = IntSlider(
    description='Day:',
    layout=Layout(width="600px"),
    min=-1, max=len(day_cols)-1, value=-1)
jslink((play, 'value'), (day_slider, 'value'))
cb = partial(slider_changed,
             the_map=m,
             # output=output,
             group=confirmed_group,
             slider=day_slider)
day_slider.observe(cb, names='value')
widget_control1 = WidgetControl(
    widget=HBox([play, day_slider]),
    position='bottomleft',
    layout=Layout(width="600px"),
)
day_slider.value = 0
day_slider.min = 0
m.add_control(widget_control1)

In [ ]:
widget_control2 = WidgetControl(
    widget=create_qrcode("https://bit.ly/3dgCR7h"),
    position='bottomleft',
    # layout=Layout(width="600px"),
)
m.add_control(widget_control2)

In [ ]:
m